In [1]:
import pandas as pd
import os
os.getcwd()

'/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data'

In [4]:
aggregated_reigster_store=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/output_sing_up_location/BL_id_by_register_store_JL_2019-04-09.csv",
                                       dtype=str,sep=",")
aggregated_reigster_store.shape

(25013950, 3)

In [5]:
aggregated_reigster_store.head(2)

,customer_id_hashed,email_address_hash,sign_up_location
0,3132829a904f1d6c44a1825bb3da6a1172c8da574a48b5...,a1fed2adf904f2b22fbdc0802904e4cb6db38fd09c42a0...,5294
1,0759b8f23ed38c541d242a6fdd905b3874978cb332321d...,353cfff59ae492fd641ae64357c46dae76d6336229f351...,5143


In [11]:
master_file_2018Q1_2019Q1=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_location'])

###
master_file_gap=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_location'])
# master_file_gap['sign_up_date'].max() is 20190601

In [17]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)
        
            
master_2019=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
master_2019=[x for x in master_2019 if "aster" in x]
df_master_2019=pd.DataFrame({"file_path":master_2019})
df_master_2019['week_end_dt']=df_master_2019['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10])
df_master_2019=df_master_2019[df_master_2019['week_end_dt']>="2019-06-02"]

master_file_later=pd.DataFrame(columns=master_file_gap.columns.tolist())
for file in df_master_2019['file_path'].tolist():
    df=pd.read_table(file,dtype=str,usecols=master_file_gap.columns.tolist(),sep="|")
    master_file_later=master_file_later.append(df)

In [18]:
aggregated_reigster_store=master_file_later.append(master_file_gap).append(master_file_2018Q1_2019Q1).append(aggregated_reigster_store)
aggregated_reigster_store=aggregated_reigster_store.drop_duplicates("customer_id_hashed")

In [19]:
aggregated_reigster_store.head(2)

,customer_id_hashed,email_address_hash,sign_up_location
0,b87f55e29bb2ab5e59e9ccd23943e410bde600fdbb7cd1...,48debf52080c9b6425529839f1d8ecda1df52328673c87...,5225
1,3d2c04fdd17cb1fe8202b0d5fda20616fa4b220682c521...,f65caadae5f2de7abe996a618df7f7c178ec4bdf3589b7...,376


In [20]:
aggregated_reigster_store.shape

(28883626, 3)

In [21]:
aggregated_reigster_store.to_csv("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/output_sing_up_location/BL_id_by_register_store_JL_2019-07-12.csv",index=False)
